## 第２章 状態空間問題

練習問題の解答

### 問題１

行動集合に斜め移動の行動が加わります。
状態集合の表現に変化はありません。
ただし、斜め移動が加わったことによって、４方向移動の時には行けなかった場所にも行けるようになるかもしれないので、*到達可能な*状態集合は変わる可能性があります。

In [ ]:
# グリッド経路探索問題 (8方向移動)
from state_space_problem import StateSpaceProblem
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

class GridState:
    def __init__(self, xy):
        self.x = xy[0]
        self.y = xy[1]

    def __hash__(self):
        return hash((self.x, self.y))

    def __eq__(self, other):
        return (self.x == other.x) and (self.y == other.y)
    
    def __str__(self):
        return "(" + self.x.__str__() + ", " + self.y.__str__() + ")"

class GridPathfinding8(StateSpaceProblem):
    state_type = GridState

    def __init__(self, 
                 width=5, 
                 height=5, 
                 init_position=(0, 0), 
                 goal_position=(4, 4)):
        
        self.width = width
        self.height = height
        self.init_position = init_position
        self.goal_position = goal_position

        self.init_state = GridState(self.init_position)

    def get_init_state(self):
        return self.init_state
    
    def is_goal(self, state):
        return (state.x == self.goal_position[0]) and (state.y == self.goal_position[1])

    def get_available_actions(self, state):
        actions = []
        if state.x > 0:
            actions.append('l')
        if state.x < self.width - 1:
            actions.append('r')
        if state.y > 0:
            actions.append('u')
        if state.y < self.height - 1:
            actions.append('d')
        
        # Diagonal actions
        if state.x > 0 and state.y > 0:
            actions.append('ul')  # Up-Left
        if state.x < self.width - 1 and state.y > 0:
            actions.append('ur')  # Up-Right
        if state.x > 0 and state.y < self.height - 1:
            actions.append('dl')  # Down-Left
        if state.x < self.width - 1 and state.y < self.height - 1:
            actions.append('dr')  # Down-Right
        return actions

    def get_next_state(self, state, action):
        if action == 'l':
            return GridState((state.x - 1, state.y))
        elif action == 'r':
            return GridState((state.x + 1, state.y))
        elif action == 'u':
            return GridState((state.x, state.y - 1))
        elif action == 'd':
            return GridState((state.x, state.y + 1))
        # Diagonal actions
        elif action == 'ul':
            return GridState((state.x - 1, state.y - 1))
        elif action == 'ur':
            return GridState((state.x + 1, state.y - 1))
        elif action == 'dl':
            return GridState((state.x - 1, state.y + 1))
        elif action == 'dr':
            return GridState((state.x + 1, state.y + 1))
        else:
            raise Exception("Invalid action: " + action)
        
    def get_action_cost(self, state, action):
        if action in ['ul', 'ur', 'dl', 'dr']:
            return 1.4  # Approximate cost for diagonal movement
        return 1

    # マンハッタン距離ヒューリスティック - グリッド経路探索
    def heuristic(self, state):
        return abs(state.x - self.goal_position[0]) + abs(state.y - self.goal_position[1])
    
    def generate_path_image(self, path_with_actions, filename="path.png"):
        """
        Generate an image of the path the agent took.
        
        Args:
            path_with_actions: A list of (state, action) pairs representing the path.
            filename: The filename to save the image to.
            
        Returns:
            The path to the saved image.
        """
        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Set the limits of the plot
        ax.set_xlim(-0.5, self.width - 0.5)
        ax.set_ylim(-0.5, self.height - 0.5)
        
        # Invert y-axis to match grid coordinates (0,0 at top-left)
        ax.invert_yaxis()
        
        # Draw grid lines
        for i in range(self.width + 1):
            ax.axvline(i - 0.5, color='gray', linestyle='-', alpha=0.5)
        for i in range(self.height + 1):
            ax.axhline(i - 0.5, color='gray', linestyle='-', alpha=0.5)
        
        # Draw start and goal positions
        ax.add_patch(patches.Rectangle(
            (self.init_position[0] - 0.5, self.init_position[1] - 0.5),
            1, 1, facecolor='green', alpha=0.3))
        ax.add_patch(patches.Rectangle(
            (self.goal_position[0] - 0.5, self.goal_position[1] - 0.5),
            1, 1, facecolor='red', alpha=0.3))
        
        # Extract states from path_with_actions
        states = [state for state, _ in path_with_actions]
        
        # Draw the path
        x_coords = [state.x for state in states]
        y_coords = [state.y for state in states]
        
        # Plot the path
        ax.plot(x_coords, y_coords, 'b-', linewidth=2, alpha=0.7)
        
        # Add markers for each step
        ax.scatter(x_coords, y_coords, color='blue', s=100, zorder=3)
        
        # Add step numbers
        for i, (x, y) in enumerate(zip(x_coords, y_coords)):
            ax.text(x, y, str(i), color='white', ha='center', va='center', fontweight='bold')
        
        # Add arrows to show direction of movement
        for i in range(len(path_with_actions) - 1):
            state, action = path_with_actions[i]
            next_state = states[i + 1]
            
            # Calculate the midpoint for the arrow
            mid_x = (state.x + next_state.x) / 2
            mid_y = (state.y + next_state.y) / 2
            
            # Calculate the direction vector
            dx = next_state.x - state.x
            dy = next_state.y - state.y
            
            # Add an arrow
            ax.arrow(mid_x - 0.3 * dx, mid_y - 0.3 * dy, 0.6 * dx, 0.6 * dy,
                    head_width=0.2, head_length=0.2, fc='blue', ec='blue', zorder=4)
        
        # Set grid
        ax.grid(True, linestyle='-', alpha=0.7)
        
        # Set title and labels
        ax.set_title('Grid Pathfinding Solution')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        
        # Add a legend
        start_patch = patches.Patch(color='green', alpha=0.3, label='Start')
        goal_patch = patches.Patch(color='red', alpha=0.3, label='Goal')
        path_line = plt.Line2D([0], [0], color='blue', linewidth=2, label='Path')
        ax.legend(handles=[start_patch, goal_patch, path_line], loc='upper right')
        
        # Save the figure
        plt.tight_layout()
        plt.savefig(filename, dpi=300)
        plt.close(fig)
        
        return filename

### 問題２

実はTSPの上界・下界の評価は様々な手法があります。ここでは簡単な例を紹介します。
より厳密な上界・下界について知りたい方は[The Traveling Salesman Problem: A Computational Study](https://www.jstor.org/stable/j.ctt7s8xg)などを参照ください。

- 下界 ($c_l$) の評価:

最小全域木 (MST): グラフの最小全域木のコストは、TSPツアーのコストの下界となります。最小全域木は、グラフのすべての頂点をつなぐ辺の重みの合計が最小となる木です。プリム法やクラスカル法を使って最小全域木を計算できます。

- 上界 ($c_h$) の評価:

最近傍法: ランダムな都市からスタートし、まだ訪れていない最も近い都市を繰り返し訪れることで、ツアーを構築します。この方法で得られたツアーのコストは、TSPの最適解の上界となります。これは、実装が簡単で、計算コストも低い方法です。

### 問題３

3x3のグリッド経路探索問題（上下左右に移動可能）における分枝度は以下の通りです。

- 角: 2方向への移動が可能。角は4箇所。
- 辺: 3方向への移動が可能。辺は4箇所。
- 中央: 4方向への移動が可能。中央は1箇所。

分枝度の平均値は、(2 * 4 + 3 * 4 + 4 * 1) / 9 = 2.67 (約) となります。

### 問題４

ブランクタイルの位置が角、辺、中央にある時の分枝度は以下の通りです（これはグリッド経路探索と一緒ですね）。

- 角: 2方向への移動が可能。角は4箇所。
- 辺: 3方向への移動が可能。辺は4箇所。
- 中央: 4方向への移動が可能。中央は1箇所。

まず、状態集合の大きさは|S| = 9! / 2 = 181,440です（[偶奇性](https://ja.wikipedia.org/wiki/15%E3%83%91%E3%82%BA%E3%83%AB)があるため半分になります）。
ブランクタイルの位置がある位置に固定された状態の集合を考えると、その大きさは|S|の1/9、つまり20,160になります。
これらの状態に対する分枝度を考えると、

- 角: 4箇所 * 2方向 * 20,160個 = 161,280個
- 辺: 4箇所 * 3方向 * 20,160個 = 241,920個
- 中央: 1箇所 * 4方向 * 20,160個 = 80,640個

となります。これらの平均を取ると、483,840 / 181,440 = 2.6666..になります。

つまり、3x3のスライディングパズルの分枝度はおよそ2.67になります。


ところで、第３，４章で紹介する探索アルゴリズムを使ってスライディングタイルを解くとき、ノードを展開した時に生成されるノードの数の平均値はこの分枝度に収束していくのでしょうか？  
良ければ考えてみてください。